In [ ]:
import cv2
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
import os
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow INFO and WARNING
os.environ['ABSL_LOG_LEVEL'] = '3'        # Suppress absl logging

print("TensorFlow Version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      print("Found GPU:", gpu)
    print(f"Avail GPU Num: {len(gpus)}")
  except RuntimeError as e:
    print(e)
else:
  print("No GPU found, using CPU...")


def load_data(image_dir, mask_dir, image_size=(128, 128)):
    if not os.path.exists(image_dir):
        raise FileNotFoundError(f"Image directory not found: {image_dir}")
    if not os.path.exists(mask_dir):
        raise FileNotFoundError(f"Mask directory not found: {mask_dir}")

    images = []
    masks = []

    # List all image filenames and sort them
    image_filenames = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])
    mask_filenames = sorted([f for f in os.listdir(mask_dir) if f.endswith('.png')])

    for filename in image_filenames:
        img_path = os.path.join(image_dir, filename)
        mask_filename = filename.replace('.jpg', '.png')  # Replace extension for masks
        mask_path = os.path.join(mask_dir, mask_filename)

        if not os.path.exists(img_path):
            print(f"Image file not found: {img_path}")
            continue
        if not os.path.exists(mask_path):
            print(f"Mask file not found: {mask_path}")
            continue

        img = load_img(img_path, color_mode='rgb', target_size=image_size)
        mask = load_img(mask_path, color_mode='grayscale', target_size=image_size)

        img = img_to_array(img)
        mask = img_to_array(mask)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Correct paths according to your setup
image_dir = "/home/starlink/bxhu/sic-unet-sat/Dataset/Image"
mask_dir = "/home/starlink/bxhu/sic-unet-sat/Dataset/Mask"

# Load the dataset
images, masks = load_data(image_dir, mask_dir)


In [ ]:
len(images)

In [ ]:
len(masks)

## Normalizing and Visualizing a sample image & mask

In [ ]:
print(np.min(images[0]), np.max(images[0]))

In [ ]:
print(images[0].shape)

In [ ]:
images[0] = images[0].astype(float)

In [ ]:
normalized_image = (images[0] - 6.0) / (255.0 - 6.0)
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(np.clip(normalized_image, 0, 1))
plt.axis('off')
plt.title("Image")

plt.subplot(1, 2, 2)
plt.imshow(np.clip(masks[0], 0, 1))
plt.axis('off')
plt.title("Mask")

plt.show()

In [ ]:

# Normalize images and masks
images = images / 255.0
masks = masks / 255.0

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)


## U-Net Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate

def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)
    
    # Encoder (downsampling path)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(c5)

    # Decoder (upsampling path)
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(32, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(32, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
model = unet_model()

# Training the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3, batch_size=1)


## Plotting Model's Loss and Accuracy

In [ ]:
plt.figure(figsize=(12, 6))

# Plot training & validation loss values
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

plt.show()

In [ ]:
# predictions = model.predict(X_val)
predictions = model.predict(X_val, batch_size=1)

In [ ]:
def plot_image_groundtruth_prediction(image, groundtruth, prediction, index):
    plt.figure(figsize=(15, 5))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(image[index])
    plt.axis('off')
    plt.title('Image')

    # Ground Truth
    plt.subplot(1, 3, 2)
    plt.imshow(groundtruth[index].squeeze(),cmap='gray')  
    plt.axis('off')
    plt.title('Ground Truth')

    # Prediction
    plt.subplot(1, 3, 3)
    plt.imshow(prediction[index].squeeze(),cmap='gray')  
    plt.axis('off')
    plt.title('Prediction')

    plt.show()

# Sample
plot_image_groundtruth_prediction(X_val, y_val, predictions, index=0)

## Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator 
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


datagen.fit(X_train)


In [ ]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=1),
                    validation_data=(X_val, y_val),
                    epochs=3)

## Predict with Data Augmentation

In [ ]:
def predict_with_augmentation(model, datagen, X_val, y_val, index=0):
    # Create a batch generator for the validation data
    augmented_generator = datagen.flow(X_val, batch_size=1, shuffle=False)
    
    # Get augmented image and mask
    augmented_image = next(augmented_generator)[0]
    
    # Make predictions
    predictions = model.predict(np.expand_dims(augmented_image, axis=0))
    
    return augmented_image, y_val[index], predictions[0]

# Example usage
augmented_image, groundtruth, prediction = predict_with_augmentation(model, datagen, X_val, y_val, index=0)


In [ ]:
def plot_image_groundtruth_prediction(image, groundtruth, prediction):
    plt.figure(figsize=(15, 5))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Image')

    # Ground Truth
    plt.subplot(1, 3, 2)
    plt.imshow(groundtruth.squeeze(), cmap='gray')  
    plt.title('Ground Truth')

    # Prediction
    plt.subplot(1, 3, 3)
    plt.imshow(prediction.squeeze(), cmap='gray')  
    plt.axis('off')
    plt.title('Prediction')

    plt.show()

# Example usage
plot_image_groundtruth_prediction(augmented_image, groundtruth, prediction)


In [ ]:
import matplotlib.pyplot as plt

def plot_image_groundtruth_prediction(images, groundtruths, predictions, index):
    plt.figure(figsize=(15, 5))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(images[index])
    plt.axis('off')
    plt.title('Image')

    # Ground Truth
    plt.subplot(1, 3, 2)
    plt.imshow(groundtruths[index].squeeze(), cmap='gray')
    plt.axis('off')
    plt.title('Ground Truth')

    # Prediction
    plt.subplot(1, 3, 3)
    plt.imshow(predictions[index].squeeze(), cmap='gray')
    plt.axis('off')
    plt.title('Prediction')

    plt.show()

# Example usage
plot_image_groundtruth_prediction(X_val, y_val, predictions, index=0)


In [ ]:
import matplotlib.pyplot as plt
import os

# --- Best Practice: Ensure the output directory exists ---
output_dir = './output/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# ---------------------------------------------------------

def plot_image_groundtruth_prediction(images, groundtruths, predictions, index):
    # Create a new figure for each plot
    fig = plt.figure(figsize=(15, 5))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(images[index])
    plt.axis('off')
    plt.title('Image')

    # Ground Truth
    plt.subplot(1, 3, 2)
    plt.imshow(groundtruths[index].squeeze(), cmap='gray')
    plt.axis('off')
    plt.title('Ground Truth')

    # Prediction
    plt.subplot(1, 3, 3)
    plt.imshow(predictions[index].squeeze(), cmap='gray')
    plt.axis('off')
    plt.title('Prediction')
    
    # Save the figure to a file
    plt.savefig(os.path.join(output_dir, f"cmp-{index}.png"))
    
    # --- IMPORTANT: Close the figure to free up memory ---
    plt.close(fig)

# --- Your existing loop remains the same ---
for idx in range(len(X_val)):
    plot_image_groundtruth_prediction(X_val, y_val, predictions, index=idx)

print("All images have been saved.")